# Зависимости



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install numpy
! pip install pymorphy2[fast]
! pip install nltk


     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 8.2 MB 24.5 MB/s 
     |████████████████████████████████| 371 kB 64.2 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=871496 sha256=b46ec96de43116548b7096d6d681d23c07b5f1a6cc3b93f3b5573e32e55bf166
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
import random
import nltk
import re
import string
import os
import shutil
from nltk.corpus import stopwords
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from typing import List
import nltk
import string
import pymorphy2
import codecs
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='pymorphy2')

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
stopWordsRu = set(stopwords.words("russian")) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [23]:
class MaskCreator():

  def __init__(self):
    self.morph = pymorphy2.MorphAnalyzer()
    self.tokenizer = nltk.WordPunctTokenizer()
    self.bigram_mod = gensim.models.Phrases.load("/content/drive/MyDrive/aiijc_sber/bigram_model.pkl")
    self.keywords = set()
    for category in ['literature', 'music', 'sport', 'animals']:
      with open("/content/drive/MyDrive/aiijc_sber/true_keywords_nouns_" + category+ ".txt", 'r') as file:
        self.keywords |= set([line.strip() for line in file.readlines()])

  def mask(self, text) -> (str, List[str]):
    mask_num = 0
    masks_dict = []
    tokens = self.tokenizer.tokenize(text)
    for token in tokens:
      word = self.morph.parse(token.lower())[0].normal_form
      if word in self.keywords:
        text = text.replace(token, 'mask' + str(mask_num))
        if word not in masks_dict:
          masks_dict.append(word)
          mask_num += 1
    return text, masks_dict

In [24]:
MaskCreator().mask("В книге рассказывается история о том, как Гном и эльф стреляли на дальность и поспорили о том, чья стрела улетит дальше. Решили, что у каждого будет по три попытки, из которых засчитывается самая лучшая. Эльф стрелял из лука, и в первый раз стрела улетела на 56 футов. Во второй раз стрела улетела на 7 футов дальше, а в третий — на 2 фута ближе. Гном стрелял из арбалета, и в первой попытке его стрела улетела на 8 футов ближе, чем в третьей попытке. Во второй попытке — на 58 футов, а в третьей — на 5 футов дальше, чем во второй. На сколько процентов отличается расстояние лучшего выстрела эльфа от расстояния лучшего выстрела гнома? Расстояние лучшего выстрела эльфа отличается на … % от расстояния лучшего выстрела гнома. Введи ответ без пробелов и единиц измерения.")

('В mask0 рассказывается история о том, как mask1 и mask2 стреляли на дальность и поспорили о том, чья стрела улетит дальше. Решили, что у каждого будет по три попытки, из которых засчитывается самая лучшая. mask3 стрелял из лука, и в первый раз стрела улетела на 56 футов. Во второй раз стрела улетела на 7 футов дальше, а в третий — на 2 фута ближе. mask1 стрелял из арбалета, и в первой попытке его стрела улетела на 8 футов ближе, чем в третьей попытке. Во второй попытке — на 58 футов, а в третьей — на 5 футов дальше, чем во второй. На сколько процентов отличается mask3 лучшего выстрела mask2а от mask4 лучшего выстрела mask4? mask4 лучшего выстрела mask2а отличается на … % от mask4 лучшего выстрела mask4. Введи ответ без пробелов и единиц mask4.',
 ['книга', 'гном', 'эльф', 'расстояние', 'измерение'])